In [1]:
from koselleck import *

In [2]:
swas=get_word_abstractness_scores()

In [3]:
# def prdz(y,ystart=1710,yend=3000,ystep=40):
def prdz(y,ystart=1600,yend=3000,ystep=20):
    ln=None
    for n in range(ystart,yend,ystep):
        if ln is None: ln=n
        if y<n: return ln
        ln=n
        
prdz(1715)

1700

In [4]:
def get_neighb_data(ystart=1600,ystep=20):
    df=pd.read_pickle('data/data.neighborhoods.cache.v2.pkl')
    df['period_int']=df.period.apply(lambda x:int(x[:4]))
    df['period2']=df.period_int.apply(lambda y: prdz(y,ystart=ystart,ystep=ystep))
    #df['period']=df['period']
    return df.query('1720<=period_int<1900')


dfres=df=get_neighb_data()
# df.groupby('period').size()
df

FileNotFoundError: [Errno 2] No such file or directory: 'data/data.neighborhoods.cache.v2.pkl'

In [5]:
dfres.period_int.describe()

count    1.720881e+07
mean     1.795133e+03
std      5.040836e+01
min      1.720000e+03
25%      1.750000e+03
50%      1.790000e+03
75%      1.840000e+03
max      1.890000e+03
Name: period_int, dtype: float64

In [6]:
dfres.loc['progress']

,period,word2,rank,count,rankcount,rank2,period_int,period2
word,,,,,,,,
progress,1720-1730,rentals,1.96,25,8.445946,2.0,1720,1720
progress,1720-1730,rental,2.40,25,7.352941,3.0,1720,1720
progress,1720-1730,conditions,1.72,25,9.191176,1.0,1720,1720
progress,1720-1730,tacks,23.00,16,0.666667,22.0,1720,1720
progress,1720-1730,hempen,10.28,25,2.216312,5.0,1720,1720
...,...,...,...,...,...,...,...,...
progress,1890-1900,mutations,33.00,1,0.029412,74.0,1890,1880
progress,1890-1900,enterprise,48.00,1,0.020408,82.5,1890,1880
progress,1890-1900,colonising,47.00,1,0.020833,80.0,1890,1880


In [13]:
dfabs=get_dfpiv_abs().reset_index().melt(id_vars=['word'],value_name='abs')
dfabs['period2']=dfabs['period'].apply(prdz)
dfabs=dfabs.groupby(['word','period2']).mean().reset_index().dropna()
dfabs['word2']=dfabs['word']
okwords=set(dfabs.word)
dfabs

,word,period2,abs,word2
4,abandonment,1780,1.249668,abandonment
5,abandonment,1800,2.563577,abandonment
6,abandonment,1820,2.896265,abandonment
7,abandonment,1840,2.677320,abandonment
8,abandonment,1860,2.750508,abandonment
...,...,...,...,...
60005,zoology,1800,0.090642,zoology
60006,zoology,1820,-0.096660,zoology
60007,zoology,1840,0.144612,zoology
60008,zoology,1860,0.079964,zoology


In [14]:
import warnings
warnings.filterwarnings('ignore')

In [19]:
def show(
        word='station',
        min_periods=2,
        max_rank=25,
        max_rank1=10,
        width=8,
        height=6,
        save=False,
        vnum='v17-20y-10-v-25-minp2',
        showdata=False,
        year_min=1720,
        year_max=1900):
    w=word
    if w in set(dfres.index):
        wdf=dfres.loc[w].query(f'period2>={year_min}')
        wdf=wdf.query(f'period2<{year_max}')
        wdf=wdf[wdf['rank2']<=max_rank]
        wdf=wdf[wdf.word2.isin(okwords)]
        wdf=wdf[~wdf.word2.isin({'foil','slate','communicating','organisation'})]
        figdf=wdf
        words_ever_high=set(wdf[wdf['rank2']<=max_rank1].word2)
        figdf=figdf[figdf.word2.isin(words_ever_high)].groupby(['word2','period2']).mean().reset_index()
        wdfp=figdf.pivot('word2','period2','rankcount')
        figdf=wdfp.reset_index().melt(id_vars=['word2'],value_name='rankcount')
        figdf=figdf.groupby('word2').filter(lambda gdf: len(gdf.dropna())>=min_periods)
        figdf=pd.concat(
            grp.assign(rank3=(-1*grp['rankcount']).rank(method='first'))
            for i,grp in figdf.groupby('period2')
        )
        figdf=figdf[figdf.rank3<=max_rank]
        figdf=figdf.sort_values('rankcount')#.set_index('word2')
#         figdf=figdf.merge(dfabs,on=['word2','period2'])
#         figdf['conc']=figdf['abs']*-1
#         figdf['is_abs']=figdf['abs']>0
        figdf['period_str']=[
            f'{x}' #'-{x+40 if x+40<1900 else 1900}'
            for x in figdf['period2']
        ]
        figdf['word']=figdf['word2']  
        if showdata: display(figdf)
        fig=start_fig(
            figdf,
            x='period_str',
            y='rank3',
            label='word',
            group='word',
#             fill='conc',
            figure_size=(width,height)
        ) + p9.geom_line(size=0.5) + p9.scale_y_reverse()
        
        first_appearances=figdf.sort_values('period_str').drop_duplicates('word')
        other_appearances=figdf.loc[set(figdf.index) - set(first_appearances.index)]
        
        fig+=p9.geom_label(size=6.5,fill='#BBBBBB',fontweight='bold',data=first_appearances)
        fig+=p9.geom_label(size=6.5,data=other_appearances)
        
        fig+=p9.scale_fill_gradient(low='white',high='#999999')
        fig+=p9.scale_color_gradient(low='white',high='#999999')
        fig+=p9.labs(
            title=f'Changing associations of "{w}"',
            x='Date of semantic model',
            y=f'Proximity to "{w}" →',
            fill='Concreteness'
        )
        if save:
            ofn=f'fig.changing_simple.{w}.{vnum}.png'
            fig.save(f'figures/{ofn}')
            fig.save(f'/home/ryan/Markdown/Drafts/TheGreatAbstraction/figures/{ofn}')
        display(fig)
        return figdf
    else:
        print(f'!! {w} not in data')
    
    
res=interact_manual(show)

interactive(children=(Text(value='station', description='word'), IntSlider(value=2, description='min_periods',…

In [9]:
# plot_word_info('crisis')

In [10]:
# plot_word_info('culture')